### SocialAL Models
# Data simulation and parameter recovery - multiple subjects
KLS 8.30.19; update 3.5.22  
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version

In [ ]:
import sys
print(sys.version)  

### Load modules

In [ ]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import itertools

## Pull in functions

In [ ]:
run common_functions.ipynb

In [ ]:
run baseline_functions.ipynb

## Functions to simulate data

In [ ]:
def select_response(prob):     
    n = random.uniform(0,1)
    if n > prob:
        response = 0
    else:
        response = 1
    return response

### Set constants

In [ ]:
recip_rates = {0: 0.93, 1:0.6, 2:0.07}

### New function to simulate data for one sub

In [ ]:
def sim_data(tn, params):
    # tn: number of trials desired
    # params: ground truth of parameters
    
    beta = params
    
    # initialize variables
    Probs = [0.5, 0.5, 0.5]
    EVs = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    
    # generate trial sequence
    trial_sequence = np.repeat([0,1,2], tn)
    random.shuffle(trial_sequence) #print(trial_sequence)
    
    trial = []
    choices = []
    responses = []
    
    for x in range(0,len(trial_sequence)):
        t = trial_sequence[x] 
        
        # Trial
        trial.append(x+1)
        
        # Make a choice
        choice = action_selection(get_action_selection_probs(beta, EVs[t]))
        choices.append(choice) 
    
        # Get a response
        recip_rate = recip_rates.get(t) 
        
        response = select_response(recip_rate) 
        responses.append(response)
    
        # then update value
        EVs[t] = update_value(Probs[t]) 

    data = {'Trial': trial, 'Beta': beta, 'Stim_Sequence': trial_sequence, 'Choice' : choices, 'Trustee_Response': responses}    
    return data

In [ ]:
#sim_data(15,[2])

### New function to sim data for multiple subjects

In [ ]:
def sim_mult_subject_data(n, tn, params):
    data = pd.DataFrame(columns= ['Trial', 'Stim_Sequence', 'Choice', 'Trustee_Response', 'Subject', 'Beta'])
    for x in range(1, n+1):
        sdt = sim_data(tn, params)
        sdt = pd.DataFrame.from_dict(sdt)
        sdt['Subject'] = [x for t in range(0,tn*3)]
        sdt['Beta'] = params
        data = pd.concat([data,sdt])
    return data

In [ ]:
#data = sim_mult_subject_data(10, 15, [2])
data

### Create param space

In [ ]:
# create parameter space
betas = [x * 2 for x in range(0,11)]
print(betas)

### Simulate data

In [ ]:
data = pd.DataFrame(columns= ['Trial', 'Stim_Sequence', 'Choice', 'Trustee_Response', 'Subject', 'Beta'])
for p in range(len(betas)):
    dt = sim_mult_subject_data(150, 15, betas[p])
    dt['Subject'] = dt['Subject'] + p*150
    data = pd.concat([data, dt])
    
data.to_csv(path_or_buf = '../../output/simulation/sim_baseline_model_data.csv', index = False)

In [ ]:
#df = data[data['Subject']==1200]
#df

### New function to fit model to multiple subjects

In [ ]:
def model_fit_mult_subject(data):
    pdt = pd.DataFrame(columns = ['Subject', 'beta'])
    a = pd.unique(data['Subject'])
    print('Number of Subs: ', len(a))
    for sub in range(1,len(a)+1):
        print('Subject: ', sub)
        df = data[data['Subject']==(sub)]
        dt = df.to_dict()
        params = model_fit(dt)
        #print(params)
        line = {'Subject': sub, 'beta': params.x[0]}
        pdt = pdt.append(line, ignore_index=True)   
    return(pdt)


In [ ]:
precover = model_fit_mult_subject(data)

In [ ]:
print(precover)

In [ ]:
#precover.to_csv(path_or_buf = '../../output/simulation/sim_baseline_model_fits.csv', index = False)